In [ ]:
import os
import torch
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw
import xml.etree.ElementTree as ET
from pathlib import Path
from tqdm import tqdm

import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms.v2 as T

# --- Configuration ---
# Set main data directory
DATA_DIR = Path("kaggle-data")
TRAIN_IMG_DIR = DATA_DIR / "train"
VAL_IMG_DIR = DATA_DIR / "val"
TEST_IMG_DIR = DATA_DIR / "test_final"

# Model parameters
NUM_CLASSES = 5 # 4 classes + 1 background
BATCH_SIZE = 4 # Keep this low for CPU training, 1 or 2 is good.
NUM_EPOCHS = 30 # A more realistic number for starting to see results.
LEARNING_RATE = 0.005

# Class mapping
CLASS_MAP = {
    "Epithelial": 1,
    "Lymphocyte": 2,
    "Macrophage": 3,
    "Neutrophil": 4,
}
# Create an inverse map for prediction
INV_CLASS_MAP = {v: k for k, v in CLASS_MAP.items()}

# --- KEY FIX (Device) ---
# Force CPU to avoid MPS bugs on M-series chips.
DEVICE = torch.device("cpu")
print(f"Using device: {DEVICE}")


# --- RLE Encoding Function (from problem description) ---
def rle_encode_instance_mask(mask: np.ndarray) -> str:
    """
    Convert an instance segmentation mask (H,W) -> RLE triple string.
    0 = background, >0 = instance IDs.
    """
    pixels = mask.flatten(order="F").astype(np.int32)
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    rle = []
    for i in range(len(runs) - 1):
        start = runs[i]
        end = runs[i + 1]
        length = end - start
        val = pixels[start]
        if val > 0:
            rle.extend([val, start, length])
    if not rle:
        return "0"
    return " ".join(map(str, rle))


# --- Custom Dataset Class (Verified Logic) ---
class NucleiDataset(Dataset):
    def __init__(self, image_dir, transforms=None):
        self.image_dir = Path(image_dir)
        self.transforms = transforms
        
        all_image_files = sorted([f for f in self.image_dir.glob("*.tif")])
        all_xml_files = sorted([f for f in self.image_dir.glob("*.xml")])

        self.image_files = []
        self.xml_files = []
        
        print(f"Filtering dataset in {image_dir}...")
        for img_path, xml_path in tqdm(zip(all_image_files, all_xml_files), total=len(all_image_files)):
            tree = ET.parse(xml_path)
            root = tree.getroot()
            
            # --- KEY FIX (Correct Hierarchy Parsing) ---
            # We need to find at least one ANNOTATION that has a class we care about
            # AND contains at least one REGION.
            has_valid_annotation = False
            
            # Loop over top-level Annotations
            for annotation in root.findall("Annotation"):
                # Find the class name for this ANNOTATION
                attribute = annotation.find("Attributes/Attribute")
                if attribute is None:
                    continue
                
                label_name = attribute.get("Name")
                if label_name in CLASS_MAP:
                    # This annotation is for a class we care about.
                    # Does it contain any regions (instances)?
                    if annotation.find("Regions/Region") is not None:
                         has_valid_annotation = True
                         break # Found a valid class with at least one region, this file is good.
            
            if has_valid_annotation:
                self.image_files.append(img_path)
                self.xml_files.append(xml_path)
        
        print(f"Found {len(self.image_files)} images with valid annotations out of {len(all_image_files)} total.")


    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        xml_path = self.xml_files[idx]
        
        # Load image
        img = Image.open(img_path).convert("RGB")
        width, height = img.size

        # Parse XML annotations
        tree = ET.parse(xml_path)
        root = tree.getroot()

        masks, labels, boxes = [], [], []
        
        # --- KEY FIX (Correct Hierarchy Parsing) ---
        # Loop over top-level Annotations
        for annotation in root.findall("Annotation"):
            # Find the class name for this ANNOTATION
            attribute = annotation.find("Attributes/Attribute")
            if attribute is None:
                continue
                
            label_name = attribute.get("Name")
            if label_name not in CLASS_MAP:
                continue
            
            # This is a class we care about, e.g., "Epithelial"
            label_id = CLASS_MAP[label_name]

            # Now, find all regions *within this annotation's Regions tag*
            for region in annotation.findall("Regions/Region"):
                vertices = []
                # Try the path from slide1.xml: Region -> Vertices -> Vertex
                vertex_nodes = region.findall("Vertices/Vertex")
                
                if not vertex_nodes:
                    # If that fails, try the other common path: Region -> Vertex
                    vertex_nodes = region.findall("Vertex")

                for vertex in vertex_nodes:
                    x = float(vertex.get("X"))
                    y = float(vertex.get("Y"))
                    vertices.append((x, y))

                if not vertices:
                    # This region has a class but no vertices, skip it.
                    continue

                # Create a binary mask for this single instance
                instance_mask = Image.new("L", (width, height), 0)
                ImageDraw.Draw(instance_mask).polygon(vertices, outline=1, fill=1)
                mask_np = np.array(instance_mask)
                
                # Get bounding box from mask
                pos = np.where(mask_np)
                if pos[0].size == 0 or pos[1].size == 0:
                    continue # Skip empty masks
                xmin, xmax = np.min(pos[1]), np.max(pos[1])
                ymin, ymax = np.min(pos[0]), np.max(pos[0])
                
                # Check for valid box area
                if xmax > xmin and ymax > ymin:
                    masks.append(mask_np)
                    labels.append(label_id)
                    boxes.append([xmin, ymin, xmax, ymax])

        if not boxes: # This should now only happen if a "valid" file has 0-area polygons
            target = {
                "boxes": torch.zeros((0, 4), dtype=torch.float32),
                "labels": torch.zeros(0, dtype=torch.int64),
                "masks": torch.zeros((0, height, width), dtype=torch.uint8)
            }
        else:
            # Convert to tensors
            boxes = torch.as_tensor(boxes, dtype=torch.float32)
            labels = torch.as_tensor(labels, dtype=torch.int64)
            masks = torch.as_tensor(np.array(masks), dtype=torch.uint8)

            target = {"boxes": boxes, "labels": labels, "masks": masks}
        
        # Apply transforms
        img_tensor = T.ToImage()(img) # Convert PIL to tensor
        img_tensor = T.ToDtype(torch.float32, scale=True)(img_tensor) # Normalize to [0,1]
        
        if self.transforms:
            # Apply augmentations if any (e.g., RandomHorizontalFlip)
            # Note: v2 transforms update target in-place
            img_tensor, target = self.transforms(img_tensor, target)

        return img_tensor, target

# --- Data Augmentation ---
def get_transforms(is_train):
    transforms = []
    # ToImage() and ToDtype() are now handled in __getitem__
    if is_train:
        # Adds random horizontal flipping for augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    
    # --- FIX: Return None if transforms list is empty ---
    if not transforms:
        return None
    
    return T.Compose(transforms)

# --- Model Definition ---
def get_model(num_classes):
    # Load a pre-trained instance segmentation model
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")

    # Replace the box predictor
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # Replace the mask predictor
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes)
    
    return model

# --- Utility for DataLoader ---
def collate_fn(batch):
    return tuple(zip(*batch))

# --- Training Loop ---
def train_one_epoch(model, optimizer, data_loader, device):
    model.train()
    loop = tqdm(data_loader, leave=True)
    total_loss = 0
    
    for images, targets in loop:
        # Filter out any 'None' targets that might result from empty images
        # This is a safety check
        valid_batch = [(img, tgt) for img, tgt in zip(images, targets) if tgt is not None]
        if not valid_batch:
            print("Skipping empty batch")
            continue
            
        images, targets = zip(*valid_batch)
        
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Skip batches that became empty after filtering (e.t., all images had 0 valid instances)
        if len(images) == 0:
            print("Skipping batch with no valid instances after processing.")
            continue
            
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        
        # Handle potential NaN losses
        if torch.isnan(losses):
            print("Warning: NaN loss detected. Skipping batch.")
            continue

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        
        total_loss += losses.item()
        loop.set_postfix(loss=losses.item())
        
    return total_loss / len(data_loader)

# --- NEW: Evaluation Function ---
def evaluate(model, data_loader, device):
    model.train() # <-- FIX: Set to train() to get loss dict, but no_grad() will stop updates
    total_loss = 0
    loop = tqdm(data_loader, leave=True, desc="Evaluating")
    
    with torch.no_grad(): # Don't calculate gradients
        for images, targets in loop:
            valid_batch = [(img, tgt) for img, tgt in zip(images, targets) if tgt is not None]
            if not valid_batch:
                continue
            
            images, targets = zip(*valid_batch)
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            if len(images) == 0:
                continue

            # During evaluation, the model still needs targets to calculate loss
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
            
            if not torch.isnan(losses):
                total_loss += losses.item()
            
            loop.set_postfix(val_loss=losses.item())

    return total_loss / len(data_loader)

# --- Main Execution Block ---
if __name__ == "__main__":
    
    # 1. Setup Datasets and DataLoaders
    print("Setting up datasets...")
    dataset_train = NucleiDataset(TRAIN_IMG_DIR, transforms=get_transforms(is_train=True))
    dataset_val = NucleiDataset(VAL_IMG_DIR, transforms=get_transforms(is_train=False))

    # --- KEY FIX (DataLoader) ---
    # Set num_workers=0 to avoid multiprocessing hangs on macOS
    train_loader = DataLoader(
        dataset_train, batch_size=BATCH_SIZE, shuffle=True, 
        num_workers=0, collate_fn=collate_fn
    )
    val_loader = DataLoader(
        dataset_val, batch_size=1, shuffle=False, 
        num_workers=0, collate_fn=collate_fn
    )

    # 2. Initialize Model, Optimizer
    print("Initializing model...")
    model = get_model(NUM_CLASSES)
    model.to(DEVICE) # Move model to CPU

    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=LEARNING_RATE, momentum=0.9, weight_decay=0.0005)
    
    # 3. Training
    print(f"Starting training for {NUM_EPOCHS} epochs on CPU. This will take a while...")
    
    # --- NEW: Variables to track the best model ---
    best_val_loss = float('inf')
    best_model_path = "nuclei_maskrcnn_model_cpu_BEST.pth"

    for epoch in range(NUM_EPOCHS):
        # --- Train for one epoch ---
        avg_train_loss = train_one_epoch(model, optimizer, train_loader, DEVICE)
        
        # --- Evaluate on the validation set ---
        avg_val_loss = evaluate(model, val_loader, DEVICE)
        
        print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")
    
        # --- NEW: Checkpoint logic ---
        # Save the model *only if* the validation loss improved
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), best_model_path)
            print(f"New best model saved with validation loss: {avg_val_loss:.4f}")

    # 4. Prediction/Inference
    print("--- Training finished ---")
    print(f"Loading best model from {best_model_path} for prediction...")
    
    # --- NEW: Load the best model's weights ---
    model.load_state_dict(torch.load(best_model_path, map_location=DEVICE))
    model.eval() # Set to evaluation mode
    
    test_files = sorted(list(TEST_IMG_DIR.glob("*.tif")))
    results = []
    
    # Get transforms for testing (just normalization)
    # --- FIX: Remove this unused and buggy line ---
    # test_transforms = get_transforms(is_train=False)

    print("Starting prediction on the test set...")
    for img_path in tqdm(test_files):
        img = Image.open(img_path).convert("RGB")
        # Apply the simple tensor conversion and normalization
        img_tensor = T.ToImage()(img)
        img_tensor = T.ToDtype(torch.float32, scale=True)(img_tensor)
        
        with torch.no_grad():
            prediction = model([img_tensor.to(DEVICE)])[0]
            
        # Initialize one instance mask per class
        instance_masks = {class_name: np.zeros((img.height, img.width), dtype=np.int32) for class_name in CLASS_MAP.keys()}
        instance_counters = {class_name: 1 for class_name in CLASS_MAP.keys()}

        # Filter predictions by score and process them
        confidence_threshold = 0.5
        scores = prediction['scores'].cpu().numpy()
        high_conf_indices = np.where(scores > confidence_threshold)[0]

        for i in high_conf_indices:
            label_id = prediction['labels'][i].item()
            if label_id not in INV_CLASS_MAP:
                continue
            
            class_name = INV_CLASS_MAP[label_id]
            
            # Get the binary mask by thresholding the soft mask
            mask = prediction['masks'][i, 0].cpu().numpy()
            binary_mask = (mask > 0.5).astype(np.uint8)
            
            # Add instance to the correct class mask with a unique ID
            instance_id = instance_counters[class_name]
            # Ensure no overlap: only assign ID where mask is 1 AND current pixel is 0
            instance_masks[class_name][(binary_mask == 1) & (instance_masks[class_name] == 0)] = instance_id
            instance_counters[class_name] += 1
            
        # RLE encode each class mask
        rle_results = {
            "image_id": img_path.stem,
            "Epithelial": rle_encode_instance_mask(instance_masks["Epithelial"]),
            "Lymphocyte": rle_encode_instance_mask(instance_masks["Lymphocyte"]),
            "Macrophage": rle_encode_instance_mask(instance_masks["Macrophage"]),
            "Neutrophil": rle_encode_instance_mask(instance_masks["Neutrophil"])
        }
        results.append(rle_results)

    # 5. Create Submission CSV
    print("Creating submission file...")
    submission_df = pd.DataFrame(results, columns=["image_id", "Epithelial", "Lymphocyte", "Neutrophil", "Macrophage"])
    submission_df.to_csv("submission_gpt2.csv", index=False)
    
    print("submission.csv created successfully!")

Using device: cpu
Setting up datasets...
Filtering dataset in kaggle-data/train...


100%|██████████| 209/209 [00:02<00:00, 70.03it/s] 


Found 209 images with valid annotations out of 209 total.
Filtering dataset in kaggle-data/val...


100%|██████████| 45/45 [00:00<00:00, 83.45it/s] 


Found 45 images with valid annotations out of 45 total.
Initializing model...
Starting training for 30 epochs on CPU. This will take a while...


...

Evaluating: 100%|██████████| 45/45 [01:44<00:00,  2.32s/it, val_loss=1.04] 


Epoch 15/30, Train Loss: 1.2189, Val Loss: 0.9961
New best model saved with validation loss: 0.9961


Evaluating: 100%|██████████| 45/45 [02:19<00:00,  3.10s/it, val_loss=1.1]  


Epoch 16/30, Train Loss: 1.2088, Val Loss: 1.0647


Evaluating:  87%|████████▋ | 39/45 [01:28<00:10,  1.76s/it, val_loss=0.811]

In [7]:
print(f"Loading best model from {best_model_path} for prediction...")
    
# --- NEW: Load the best model's weights ---
model.load_state_dict(torch.load(best_model_path, map_location=DEVICE))
model.eval() # Set to evaluation mode

test_files = sorted(list(TEST_IMG_DIR.glob("*.tif")))
results = []

# Get transforms for testing (just normalization)
# --- FIX: Remove this unused and buggy line ---
# test_transforms = get_transforms(is_train=False)
cnt = 1
print("Starting prediction on the test set...")
for img_path in tqdm(test_files):
    img = Image.open(img_path).convert("RGB")
    # Apply the simple tensor conversion and normalization
    img_tensor = T.ToImage()(img)
    img_tensor = T.ToDtype(torch.float32, scale=True)(img_tensor)
    
    with torch.no_grad():
        prediction = model([img_tensor.to(DEVICE)])[0]
        
    # Initialize one instance mask per class
    instance_masks = {class_name: np.zeros((img.height, img.width), dtype=np.int32) for class_name in CLASS_MAP.keys()}
    instance_counters = {class_name: 1 for class_name in CLASS_MAP.keys()}

    # Filter predictions by score and process them
    confidence_threshold = 0.3
    scores = prediction['scores'].cpu().numpy()
    high_conf_indices = np.where(scores > confidence_threshold)[0]

    if cnt == 1:
        print(scores)
        print(high_conf_indices)
        cnt += 1

    for i in high_conf_indices:
        label_id = prediction['labels'][i].item()
        if label_id not in INV_CLASS_MAP:
            continue
        
        class_name = INV_CLASS_MAP[label_id]
        
        # Get the binary mask by thresholding the soft mask
        mask = prediction['masks'][i, 0].cpu().numpy()
        binary_mask = (mask > 0.5).astype(np.uint8)
        
        # Add instance to the correct class mask with a unique ID
        instance_id = instance_counters[class_name]
        # Ensure no overlap: only assign ID where mask is 1 AND current pixel is 0
        instance_masks[class_name][(binary_mask == 1) & (instance_masks[class_name] == 0)] = instance_id
        instance_counters[class_name] += 1
        
    # RLE encode each class mask
    rle_results = {
        "image_id": img_path.stem,
        "Epithelial": rle_encode_instance_mask(instance_masks["Epithelial"]),
        "Lymphocyte": rle_encode_instance_mask(instance_masks["Lymphocyte"]),
        "Macrophage": rle_encode_instance_mask(instance_masks["Macrophage"]),
        "Neutrophil": rle_encode_instance_mask(instance_masks["Neutrophil"])
    }
    results.append(rle_results)

# 5. Create Submission CSV
print("Creating submission file...")
submission_df = pd.DataFrame(results, columns=["image_id", "Epithelial", "Lymphocyte", "Neutrophil", "Macrophage"])
submission_df.to_csv("submission_gpt2.csv", index=False)

print("submission.csv created successfully!")

Loading best model from nuclei_maskrcnn_model_cpu_BEST.pth for prediction...
Starting prediction on the test set...


  2%|▎         | 1/40 [00:02<01:39,  2.54s/it]

[0.3542218  0.3094444  0.2953994  0.26057822 0.25756732 0.25456735
 0.24357812 0.23596412 0.21294756 0.20934927 0.20243208 0.18628149
 0.17518887 0.17496729 0.17235251 0.1711742  0.17052142 0.16467665
 0.16310106 0.14735453 0.1451866  0.13717528 0.12895611 0.12654467
 0.12573816 0.12520695 0.12473088 0.12405452 0.12377945 0.12358025
 0.12343706 0.11802054 0.11286068 0.10821794 0.10610596 0.10391385
 0.09737283 0.09650794 0.09600981 0.09500112 0.0914961  0.0913388
 0.09081868 0.08822686 0.08699933 0.08644167 0.0856558  0.08548825
 0.08539207 0.08523829 0.08378797 0.08303632 0.08275808 0.08264152
 0.08226836 0.07933556 0.07885344 0.07734136 0.07661338 0.07502476
 0.07282703 0.07195829 0.07154873 0.06873851 0.06854381 0.06807977
 0.0668014  0.06679291 0.06579588 0.06498147 0.06478667 0.06407136
 0.06369562 0.0636515  0.06335458 0.06254822 0.06254367 0.06210437
 0.06149235 0.06096388 0.05958416 0.05733868 0.05703201 0.05657746
 0.05642383 0.05620978 0.05618202 0.05609224 0.05560296 0.05529

100%|██████████| 40/40 [00:56<00:00,  1.42s/it]

Creating submission file...
submission.csv created successfully!
